# Trabajo en los modulos de la primera parte
Este Jupyter-Notebook contempla la creación y prueba del conjunto de módulos para la clasificación de posibles candidatos a ser estrellas variables de largo periodo.
## 1 Funciones del módulo de adquisición de datos
El objetivo de este módulo es proveer de las herramientas para llevar los datos de un archivo bruto con los datos de las estrellas a una estructura de tipo Astropy Table.

In [23]:
"""Data acquisition module.

The sets of functions here help to gather the data from the source.

Authors: 
"""

import re
import numpy as np
from astropy.table import Table, vstack
import datetime

def cut(c):
    """(private) Cut the data by a patron defined and return a tuple.

    Keyword arguments:
    data -- the data to cut, list
    
    """
    flag = True
    data = []
    head = []
    patron = re.compile("\s+")
    for i in c:
        if flag == True:
            head.append(patron.split(i))
            flag = False
        else:
            data.append(patron.split(i))
    return np.array(data),head

def parser(filepath, n=10):
    """Transform data from source txt to an structure and return a astropy.table.

    Keyword arguments:
    filepath -- the data's path to transform, string.
    
    """
    
    file = open(filepath,"r")
    data = file.readlines()
    file.close()
    c = []
    alpha = []
    gamma = []
    flag = 0
    flag1 = False
    typ= ('f8','f8','f8','f8','f8','f8','f8','f8','f8','f8','f8',np.str,'f8',np.str,np.str)
    for i in range(len(data)):
        flag1= re.findall("\# ######### LIGHT ",data[i])
        if flag1:
            data= data[i+1:]
            break
    if len(data) == 0:
        return -1
    try:
        for i in range(len(data)):
            head = re.findall("\A#\s+(.+)",data[i])
            new_line = re.findall("\A   (.+)",data[i]) 
            ra = re.findall("\#ra=\s+(.+)",data[i])
            dec = re.findall("\#dec=\s+(.+)",data[i])

            if ra:
                ra_aux = ra
            if dec:
                dec_aux = dec
            if head:
                [c.append(j) for j in head]
            if new_line:
                [c.append(k) for k in new_line]
                alpha.append(ra_aux)
                gamma.append(dec_aux)
                try:
                    next_line = re.findall("\A   (.+)",data[i+1])
                    if not next_line:
                        reg, col= cut(c)
                        if len(reg) == len(alpha)+1:
                            log = open("parser_log.txt", "a+")
                            reg = reg[1:len(reg)+1]
                            log.write(str(datetime.datetime.now())+" len_reg_not_equal_len_alpha "+filepath+"\n")
                            log.close()
                        if flag == 0:
                            col[0].append('RA')
                            col[0].append('DEC')
                            reg = np.c_[reg,np.array(alpha),np.array(gamma)]
                            t = Table(reg, names=tuple(col[0]), dtype = typ)
                            c = []
                            alpha = []
                            gamma = []
                            flag += 1
                        else:
                            col[0].append('RA')
                            col[0].append('DEC')
                            reg = np.c_[reg,np.array(alpha),np.array(gamma)]
                            t_aux = Table(reg, names=tuple(col[0]), dtype = typ)
                            t = vstack([t, t_aux])
                            c = []
                            alpha = []
                            gamma = []

                except:
                    reg, col = cut(c)
                    if flag == 0:
                        col[0].append('RA')
                        col[0].append('DEC')
                        reg = np.c_[reg,np.array(alpha),np.array(gamma)]
                        t = Table(reg, names=tuple(col[0]), dtype = typ)
                        c = []
                        alpha = []
                        gamma = []
                        flag += 1   
                    else:
                        col[0].append('RA')
                        col[0].append('DEC')
                        reg = np.c_[reg,np.array(alpha),np.array(gamma)]
                        t_aux = Table(reg, names=tuple(col[0]), dtype = typ)
                        t = vstack([t, t_aux])
        if (len(t)>=n):
            return t
        else:
            log = open("parser_log.txt", "a+")
            log.write(str(datetime.datetime.now())+" not_n_enough_samples "+filepath+"\n")
            log.close()
            return Table()
    except:
        log = open("parser_log.txt", "a+")
        log.write(str(datetime.datetime.now())+" unknown_error_in "+filepath+"\n")
        log.close()
        return Table()

## 2 Funciones del módulo de preprocesamiento
Acá se encuentran el conjunto de funciones que ayudarán a los datos en formato Astropy Table a limpiarlos de errores de medición.

In [24]:
"""Preprocessing module.

The sets of functions here help from the perspective of preprocessing, complementing
the module of data gathering and features extraction.

Authors: 
"""

import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table

def outliers_iqr(data_col, aperture='0'):
    """Delete atypical data and return a astropy.table.

    Keyword argument:
    ys -- the data, {np.narray, astropy.table, astropy.table.column}
    aperture -- the aperture index, str
    
    """
    if (type(data_col)==Table):
        quartile_1, quartile_3 = np.percentile(data_col['MAG_'+aperture], [25,75])
        iqr = quartile_3 - quartile_1
        lower_bound = quartile_1 - (iqr*1.5)
        upper_bound = quartile_3 + (iqr*1.5)
        res = data_col[np.where((data_col['MAG_'+aperture] <= upper_bound) & (data_col['MAG_'+aperture] >= lower_bound))]
        
    else:
        quartile_1, quartile_3 = np.percentile(data_col, [25, 75])
        iqr = quartile_3 - quartile_1
        lower_bound = quartile_1 - (iqr*1.5)
        upper_bound = quartile_3 + (iqr*1.5)
        res = data_col[np.where((data_col <= upper_bound) & (data_col >= lower_bound))]
        if (type(data_col)==Table.Column):
            res = Table([res], names=(data_col.name,), dtype=('f8',))
            res.meta = data_col.meta
        else:
            res = Table([res], names=('MAG_'+'i', ))
    return res
        
def high_photometric_errors(data_col, aperture='0'):
    """Delete high photometric errors and return a astropy.table.

    Keyword argument:
    data -- the data to analyze, {np.narray, astropy.table, astropy.table.column}
    aperture -- the aperture index, str
    
    """
    if (type(data_col)==Table):    
        mer_mean = np.mean(data_col['MER_'+aperture])
        mer_std = np.std(data_col['MER_'+aperture])
        error_limit = mer_mean + 3*mer_std
        res = data_col[np.where(data_col['MER_'+aperture] < error_limit)]

    else:
        mer_mean = np.mean(data_col)
        mer_std = np.std(data_col)
        error_limit = mer_mean + 3*mer_std
        res = data_col[np.where(data_col < error_limit)]
        if (type(data_col)==Table.Column):
            res = Table([res], names=(data_col.name,), dtype=('f8',))
            res.meta = data_col.meta
        else:
            res = Table([res], names=('MER_'+'i', ))
    return res

# funcion que hace todo el preprocesamiento
def preprocessing(data,aperture):
    """Preprocess data for an aperture and return a astropy.table.

    Keyword arguments:
    data -- the data to analyze, {np.narray, astropy.table, astropy.table.column}
    aperture -- the aperture index, str
    
    """
    # 1 - se eliminan las mediciones con alto error fotometrico
    data_aux = high_photometric_errors(data,aperture)
    #2 - se eliminan datos atípicos
    data_aux = outliers_iqr(data_aux,aperture)
    # Se retornan los dias julianos y la magnitud de la apertura seleccionada
    return data_aux


## 3.- Funciones del módulo de extracción de características
Tienen por objetivo ayudar a extraer propiedades que sirven para la toma de decisión más adelante de los datos limpios, es decir, ya preprocesados.

In [3]:
"""Features extraction module.

The objective of this module is analyze the preprocessed light curves.

Authors: 
"""

from scipy.optimize import curve_fit
import numpy as np
from astropy.table import unique

def lineal_fit(t,a,b):
    """Lineal fit to data and return param m np.narray.

    Keyword arguments:
    t -- A np.narray
    a -- A float
    b -- A float
    
    """
    m = a+b*t
    return m

def parabolic_fit(t,a,b,c):
    """Parabolic fit to data and return param m np.narray.

    Keyword arguments:
    t -- A np.narray
    a -- A float
    b -- A float
    c -- A float
    
    """
    m = a + b*t + c*t*t
    return m

# Por ahora fit puede ser lineal o parabolico
def get_statistics(t,y):
    """Obtain and return Q1', C1, Q2' and C2.

    Keyword arguments:
    t -- A np.narray, astropy.table.column
    y -- A np.narray, astropy.table.column
    
    """
    # Desviacion estandar y
    dep = np.std(y)
    #para fit lineal
    poptl, pcovl = curve_fit(lineal_fit, t, y)
    y_hatl = lineal_fit(t,*poptl)
    perrl = np.sqrt(np.diag(pcovl))
    defl = np.sqrt(np.sum((y_hatl- y)*(y_hatl- y))/len(y))
    q1 = poptl[1]/perrl[1]
    c1 = 1-(defl/dep)
    # Para fit parabolico
    poptp, pcovp = curve_fit(parabolic_fit, t, y)
    y_hatp = parabolic_fit(t,*poptp)
    perrp = np.sqrt(np.diag(pcovp))
    defp = np.sqrt(np.sum((y_hatp- y)*(y_hatp- y))/len(y))
    q2 = poptp[2]/perrp[2]
    c2 = 1-(defp/defl)
    stat = [q1,c1,q2,c2]
    return stat

def get_ra_dec(data):
    """Obtain and return RA and DEC, both str.
    
    RA is the Right Ascencion.
    DEC is the Declination.

    Keyword arguments:
    data -- the data to analyze, astropy.table
    
    """
    ra= data['RA'][0].split(" ")[0]
    dec= data['DEC'][0].split(" ")[0]
    return ra,dec

def select_unique(data,cols):
    """Obtain and return RA and DEC, both str.

    RA is the Right Ascencion.
    DEC is the Declination.

    Keyword arguments:
    data -- the data to analyze, astropy.table
    
    """

    res = unique(data,  keys=cols)
    row = []
    for i in range(len(res)):
        ra = float(res['RA'][i].split(" ")[0])
        dec = float(res['DEC'][i].split(" ")[0])
        row.append([ra,dec])
    return np.array(row)

def select_unique2(data,cols):
    """Obtain and return RA and DEC, both str.

    RA is the Right Ascencion.
    DEC is the Declination.

    Keyword arguments:
    data -- the data to analyze, astropy.table
    
    """

    res = unique(data,  keys=cols)
    row = []
    for i in range(len(res)):
        ra = res['RA']
        dec = res['DEC']
        row.append([ra,dec])
    return np.array(row)

## 4.- Funciones del módulo de filtros
Estas funciones son útiles para filtrar datos en base a umbrales o grados de error.

In [4]:
"""Filter module.

The objective of this module is to classify each star by his light curve's
params obtained from features extraction module, filtering by a group of 
thresholds the stars that are not periodic large variable.

Authors: 
"""
from astropy.table import Table,join
import numpy as np

def grade_filter(data,grades):
    """Filter the data to preprocess and return a astropy.table.

    Keyword arguments:
    data -- the data to analyze, astropy.table
    grades -- the grades to analyze, tuple
    
    """
    data_aux = Table([grades], names=['GRADE'])
    res = join(data, data_aux, join_type='right')
    if (len(res)) == 0:
        print("there's no values asociated with "+str(grades)+" apertures, no table returned")
        return -1
    if res.masked:
        del res[res['HJD'].mask.nonzero()[0]]
    return res

def stars_Cfilter(data, ThC1=0.02, ThC2=0.02):
    """Filter the data by thresholds and return the filtered data as astropy.table.
    
    ThreshC1 -- the threshold for C1
    ThreshC2 -- the threshold for C2
    
    """
    return data[np.where((data['C1']>=ThC1)&(data['C2']>=ThC2))]
    
def stars_Qfilter(data, ThQ1=4, ThQ2=4):
    """Filter the data by thresholds and return the filtered data as astropy.table.
    
    ThreshC1 -- the threshold for C1
    ThreshC2 -- the threshold for C2
    
    """
    return data[np.where((data['Q1']>=ThQ1)&(data['Q2']>=ThQ2))]


## 5.- Módulo privado de funciones varias
Uso personal en el desarrollo de este trabajo

In [5]:
""" Utilities

"""
%matplotlib notebook
from astropy import units as u
from astropy.coordinates import SkyCoord
    
def _Sort(tup):
    """Sort a tuple in descending order and return a tuple.

    Keyword arguments:
    tup -- tuple to sort, tuple
    
    """
    # reverse = True (Sorts in Descending order)
    return(sorted(tup, key = lambda x: float(x[0]), reverse = True))
        
def _graph(data,path,title):
    """Graph in a scatter type and save to a folder.

    Keyword arguments:
    data -- the data to graph, tuple
    path -- the path to save, str
    title -- the graph title, str
    
    """
    data = Sort(data)
    fig, ax = plt.subplots()
    ax.scatter(data[0],data[1])
    ax.set(xlabel='HJD', ylabel='Magnitude',
           title=title)
    ax.grid()
    fig.savefig(path+"/"+title+"_fig.png")
    
def grades():
    """Ask for grades to keyboard input and return a list with the keyboard input."""
    try:
        grades = input('Select GRADE, usage "A,B,..." with valid options A,B,C,D: ')
        if not(set(grades.split(",")) <= {'A','B','C','D'}):
            raise ValueError(grades)
        return grades.split(",")
    except ValueError:
        print("incorrect GRADE, ingressed: ",grades)
        raise
        
def mags():
    """Ask for magnitudes to keyboard input and return a tuple."""
    try:
        mags = input('Select MAG, usage MAG_2,MAG_0,MAG_1,MAG_3,MAG_4 like 0,0,1,0,0 for only MAG_1 with full weight: ')
        if (int(max(set(mags.split(",")))) > 1):
            raise ValueError(mags)
        return tuple(mags.split(","))
    except ValueError:
        print("incorrect MAG, ingressed: ",mags)
        raise

###########################Example function - blue box#############################
# usage: 
# path = directory path with data
# ThreshQ1, ThreshC1, ThreshQ2, ThreshC2 are the threshold used in final stage
# optional: when "stop" archives were opened -> stop, if 0 open all archives in path
def blue_box(path,ThreshQ1, ThreshC1, ThreshQ2, ThreshC2,stop=0):
    "Doc"
    grad = grades()
    mag = mags()
    counter = 0
    row= []
    col= ["ASAS NAME","RA","DEC","Q1","C1","Q2","C2"]
    
    for filename in os.listdir(path):
        file = open(path+filename,"r")
        text = file.readlines()
        df = parser(text)
        file.close()
        res = grade_filter(df,grad,mag)
        counter+=1
        #graph(res,"test",filename)
        stats = get_statistics(res[0],res[1])
        ra,dec= get_ra_dec(df)
        print(type(get_ra_dec(df)))
        row.append([filename,ra,dec,stats[0],stats[1],stats[2],stats[3]])
        
        if ((stop>0)&(stop==counter)):
            break
    data = pd.DataFrame(row,columns=col)
    data = stars_filter(data, ThreshQ1, ThreshC1, ThreshQ2, ThreshC2)
    return data

####################################################################################

def display_RA_DEC(data,name,mode):
    """ Display RA vs DEC in two graph, saving it to ../graficos/data_<name>.png and
    ../graficos/RA_DEC_H_<name>.png
    args:
        data(astropy.table.table.Table): The data
        name(str): The desired name of the file
        mode(int): 0 for save and 1 for show
    """
    coord = select_unique(data,['RA', 'DEC'])
    c = SkyCoord(ra=coord[:,0]*u.hour, dec=coord[:,1]*u.degree, frame='icrs')
    ra = coord[:,0] # in hours
    dec = coord[:,1] # in degrees
    #ra_rad = ra*2*np.pi/24*u.radian
    #ra_rad[ra_rad > np.pi*u.radian] -= 2. * np.pi
    #dec_rad = dec*np.pi/180*u.radian
    ra_rad = c.ra.radian
    ra_rad[ra_rad > np.pi] -= 2. * np.pi
    plt.figure(figsize=(8,4.2))
    plt.subplot(111, projection="aitoff")
    plt.title("Dataset: "+name)
    plt.grid(True)
    plt.plot(ra_rad,c.dec.radian, 'o', markersize=2, alpha=0.3)
    plt.subplots_adjust(top=1,bottom=0.0)
    plt.savefig("../graficos/data_"+name+".png")
    plt.figure()
    plt.scatter(c.ra.hour, c.dec.degree, s=2)
    plt.title("Dataset "+name)
    plt.ylabel("DEC $[°]$")
    plt.xlabel("RA $[H]$")
    plt.savefig("../graficos/DEC_RA_"+name+".png")




### Prueba de los módulos anteriores
Contempla la realización de toda la primera parte del pipeline.
#### 1.- Parser, filtrado por grade y preprocessing

In [20]:
import os
import matplotlib.pyplot as plt
import pickle

typ = ('f8', 'f8', 'f8', 'f8', 'f8', 'f8')
cols = ('RA', 'DEC', 'Q1prim', 'C1', 'Q2prim', 'C2')
for i in range(0,24):
    num = "0"+str(i) if i<10 else str(i)
    path = "../"+num+"/"
    rows = []
    flag = 0
    for filename in os.listdir(path):
        try:
            if not (os.stat(path+filename).st_size == 0):
                t = parser(path+filename,10)
                if len(t)!=0:
                    t_aux = grade_filter(t,("A","B"))
                    t_aux = preprocessing(t,'0')
                    #stats = get_statistics(t_aux['HJD'], t_aux['MAG_0'])
                    #rows = get_ra_dec(t)
                    if flag == 0:
                        data = Table(np.append([float(rows[0]), float(rows[1])], stats[:]), names=cols, dtype = typ)
                        flag = 1
                    else:
                        data_aux = Table(np.append([float(rows[0]), float(rows[1])], stats[:]), names=cols, dtype = typ)
                        data = vstack([data, data_aux])
            else:
                log = open("parser_log.txt", "a+")
                log.write(str(datetime.datetime.now())+" error_empty_file "+filename+"\n")
                log.close()
        except:
            log = open("parser_log.txt", "a+")
            log.write(str(datetime.datetime.now())+" error_reading "+filename+"\n")
            log.close()
            print("Error reading: "+filename)
    pickle.dump( data, open( "data_tables/"+num+".p", "wb" ) )

/home/fabricio/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:54: RuntimeWarning: divide by zero encountered in double_scalars
/home/fabricio/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:54: RuntimeWarning: divide by zero encountered in double_scalars
/home/fabricio/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:54: RuntimeWarning: divide by zero encountered in double_scalars
/home/fabricio/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:54: RuntimeWarning: divide by zero encountered in double_scalars
/home/fabricio/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:54: RuntimeWarning: divide by zero encountered in double_scalars
/home/fabricio/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:54: RuntimeWarning: divide by zero encountered in double_scalars
/home/fabricio/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:54: RuntimeWarning: divide by zero encountered in double_scalars
/home/fabricio/anaco

#### 2.- Filtro de los datos por parámetros $C_1$ y $C_2$

In [60]:
data

RA,DEC,Q1prim,C1,Q2prim,C2
float64,float64,float64,float64,float64,float64
23.633457,-7.941902,-1.8332295509722178,0.005554604576573641,-0.5687258505769096,0.000540484857758261
23.193289,-66.433087,-1.7714401720193715,0.0034607755390667627,3.6687531299351916,0.014627939430294723
23.10842,-13.75259,-0.6439683665728907,0.0005629959116690841,0.05042710769604924,3.4639349524301366e-06
23.379693,-30.120198,1.0455035205317789,0.0012865280851244165,-1.122768888894345,0.0014867602847070671
23.607914,11.006631,2.4224800609015156,0.01200606361860479,3.4983020481446307,0.02465964676326715
23.586119,-17.823506,1.5569910741581052,0.003535721685326787,0.5143454333425497,0.0003888045664635076
23.649768,-38.506813,3.2525823551757687,0.008101464923496615,-1.7115186269712623,0.0022665708608999857
23.476104,-67.397533,-0.2280099583712933,3.087950920199756e-05,-2.7314087002679894,0.004406265053423897
23.58303,13.734414,-0.34453966373301603,0.00039813568297952795,-0.050792709180126204,8.720806148998506e-06


In [36]:
aux = parser("../00/000006+2553.2",10)
t_aux = grade_filter(t,("A","B"))
t_aux = preprocessing(t,'0')
stats = get_statistics(t_aux['HJD'], t_aux['MAG_0'])

In [56]:
rows = select_unique2(t_aux,['RA','DEC'])

In [59]:
rows[0]

array([['23.272979  23:16:22.7', '23.273030  23:16:22.9',
        '23.273118  23:16:23.2', '23.273240  23:16:23.7',
        '23.273262  23:16:23.7', '23.273276  23:16:23.8',
        '23.273308  23:16:23.9', '23.273535  23:16:24.7'],
       ['-23.150353 -23:09:01.3', '-23.150506 -23:09:01.8',
        '-23.144755 -23:08:41.1', '-23.147471 -23:08:50.9',
        '-23.146792 -23:08:48.5', '-23.147412 -23:08:50.7',
        '-23.148244 -23:08:53.7', '-23.151006 -23:09:03.6']], dtype='<U22')

In [58]:
t_aux

HJD,MAG_0,MAG_1,MAG_2,MAG_3,MAG_4,MER_0,MER_1,MER_2,MER_3,MER_4,GRADE,FRAME,RA,DEC
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str22,float64,str22,str22
2140.70574,12.583,12.609,12.633,12.662,12.708,0.064,0.038,0.036,0.041,0.048,A,30211.0,23.273262 23:16:23.7,-23.146792 -23:08:48.5
2151.62325,12.588,12.479,12.373,12.242,12.224,0.083,0.051,0.053,0.068,0.083,D,31379.0,23.273262 23:16:23.7,-23.146792 -23:08:48.5
2487.75357,12.797,12.824,12.829,12.835,12.813,0.056,0.053,0.046,0.05,0.05,B,13531.0,23.273535 23:16:24.7,-23.151006 -23:09:03.6
2191.61104,12.686,12.616,12.61,12.639,12.665,0.05,0.037,0.03,0.031,0.032,A,35102.0,23.273276 23:16:23.8,-23.147412 -23:08:50.7
2193.59114,12.676,12.681,12.714,12.77,12.798,0.049,0.039,0.031,0.032,0.033,A,35450.0,23.273276 23:16:23.8,-23.147412 -23:08:50.7
2194.58427,12.759,12.816,12.776,12.865,12.9,0.051,0.043,0.036,0.037,0.038,A,35626.0,23.273276 23:16:23.8,-23.147412 -23:08:50.7
2195.5843,12.622,12.565,12.561,12.571,12.616,0.046,0.036,0.028,0.029,0.03,A,35803.0,23.273276 23:16:23.8,-23.147412 -23:08:50.7
2197.58756,12.617,12.667,12.691,12.69,12.64,0.041,0.037,0.028,0.03,0.03,A,36121.0,23.273276 23:16:23.8,-23.147412 -23:08:50.7
2198.57967,12.499,12.47,12.522,12.559,12.581,0.045,0.036,0.028,0.029,0.031,A,36289.0,23.273276 23:16:23.8,-23.147412 -23:08:50.7
